In [1]:
import os
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from typing import List
import numpy as np

from utils import load_oilspills

In [2]:
data_dir = '../data'

oilspills_csvpath = os.path.join(data_dir, 'US_oilspills.csv')
US_shapely_filepath = os.path.join(data_dir, 'states_21basic/states.shp')

In [3]:
# load in data
oilspills = load_oilspills(oilspills_csvpath)
oilspills.head()

,open_date,name,location,lat,lon,threat,commodity,max_ptl_release_gallons,description
id,,,,,,,,,
10049,2020-02-27,Partially Submerged Recreational Vessel Huron ...,"Huron, OH",41.391532,-82.554591,Oil,NaN,NaN,"On February 27, 2020, SSC received notificatio..."
10050,2020-02-27,Whitney Tank Battery 160 Loomis Pass Spill,"Venice, LA, USA",29.116625,-89.180917,Oil,NaN,NaN,"On February 27, 2020, the USCG Sector New Orle..."
10048,2020-02-24,Gray whale carcass,"Port Hueneme, CA",34.149744,-119.208226,Other,NaN,NaN,"On 24-FEB-2020, NMFS Stranding Coordinator in ..."
10047,2020-02-20,Recreational vessel sunk in Anacortes Skyline ...,"2400 Skyline Way, Anacortes, WA 98221, USA",48.492559,-122.680368,Oil,NaN,NaN,"On February 20, 2020, the SEA WOLF, a 44 foot ..."
10046,2020-02-17,North Santiam River - Truck Spill,"N Santiam Hwy, Detroit, OR 97342, USA",44.697000,-122.226890,Oil,NaN,NaN,On the morning of 16 FEB 2020 along the Santi...


### Task 1: Determine the number of Incidents that occured within US boundary

For each incident, determine if it is within any US state. If so, it counts as within US boundary

In [4]:
# geometry points are lon lat
us_gdf = gpd.read_file(US_shapely_filepath)
us_gdf.head()

,STATE_NAME,DRAWSEQ,STATE_FIPS,SUB_REGION,STATE_ABBR,geometry
0,Hawaii,1,15,Pacific,HI,"MULTIPOLYGON (((-160.07380 22.00418, -160.0497..."
1,Washington,2,53,Pacific,WA,"MULTIPOLYGON (((-122.40202 48.22522, -122.4628..."
2,Montana,3,30,Mountain,MT,"POLYGON ((-111.47543 44.70216, -111.48080 44.6..."
3,Maine,4,23,New England,ME,"MULTIPOLYGON (((-69.77728 44.07415, -69.85993 ..."
4,North Dakota,5,38,West North Central,ND,"POLYGON ((-98.73044 45.93827, -99.00683 45.939..."


In [5]:
def count_incidents_within_polygons(incidents_coordinates: np.ndarray, polygons: List) -> int:
    """Counts the total number of incidents within a set of polygons
    Args:
        - incidents_coordinates: (2xn) array of (lon, lat)
        - polygons: List of Shapely Polygon types
    Returns:
        - num_incidents within polygons
    """
    total_incidents_within_polygons = 0
    for incident_coord in incidents_coordinates:

        incident_geopoint = Point(*incident_coord) 

        for polygon in polygons:

            if polygon.contains(incident_geopoint):
                total_incidents_within_polygons +=1
                continue
                
    return total_incidents_within_polygons

In [6]:
oilspill_coordinates = oilspills[['lon', 'lat']].values
states_polygons = us_gdf['geometry'].values

incidents_within_states = count_incidents_within_polygons(oilspill_coordinates, states_polygons)

print(f'There are {incidents_within_states} incidents within the states out of a total of {len(oilspills)} incidents')

There are 1378 incidents within the states out of a total of 3709 incidents


### Task 2: Determine the number of incidents within Texas

In [7]:
def count_incidents_within_polygon(incidents_coordinates: np.ndarray, 
                                   polygon: gpd.array.GeometryArray) -> int:
    """Counts the total number of incidents within a single polygon.
    Args:
        - incidents_coordinates: (2xn) array of (lon, lat)
        - polygon: Shapely Polygon
    Returns:
        - num_incidents within polygon
    """
    
    total_incidents_within_polygon = 0 
    for incident_coord in incidents_coordinates:

        incident_geopoint = Point(*incident_coord) 
        
        if polygon.contains(incident_geopoint):
            total_incidents_within_polygon +=1
            
    return total_incidents_within_polygon

                

In [8]:
texas_polygon = us_gdf[us_gdf['STATE_NAME']=='Texas']['geometry'].values

incidents_within_texas = count_incidents_within_polygon(oilspill_coordinates, texas_polygon)

print(f'There are {incidents_within_texas} incidents within Texas')

There are 100 incidents within Texas


### Task 3: Identify the state with more than 50 incidents, report the state name and number

Q: Is it more than one state?

In [9]:
state_incident_counts = [count_incidents_within_polygon(oilspill_coordinates, state_polygon) for state_polygon in states_polygons]

# get indexes of states with more than 50 incidents
statess_idx_more_50_incidents= np.where(np.array(state_incident_counts)>50)[0]
statess_idx_more_50_incidents

array([ 1, 14, 16, 20, 24, 40, 47, 50])

In [10]:
states_more_50_incidents_df = us_gdf.iloc[statess_idx_more_50_incidents, :]
states_more_50_incidents_df

,STATE_NAME,DRAWSEQ,STATE_FIPS,SUB_REGION,STATE_ABBR,geometry
1,Washington,2,53,Pacific,WA,"MULTIPOLYGON (((-122.40202 48.22522, -122.4628..."
14,Massachusetts,15,25,New England,MA,"MULTIPOLYGON (((-71.31933 41.77220, -71.33980 ..."
16,New York,17,36,Middle Atlantic,NY,"MULTIPOLYGON (((-79.76324 42.26733, -79.44402 ..."
20,New Jersey,21,34,Middle Atlantic,NJ,"POLYGON ((-75.48928 39.71486, -75.47597 39.720..."
24,California,25,06,Pacific,CA,"MULTIPOLYGON (((-121.66522 38.16929, -121.7823..."
40,Texas,41,48,West South Central,TX,"MULTIPOLYGON (((-105.99889 31.39394, -106.2132..."
47,Louisiana,48,22,West South Central,LA,"MULTIPOLYGON (((-93.70752 30.23958, -93.69938 ..."
50,Alaska,51,02,Pacific,AK,"MULTIPOLYGON (((-161.33379 58.73325, -161.3824..."


### Task 4: Identify the pair of incident nearest-neighbours that has the longest distance, indicate also the distance, while the incidents occurred within/on the US states/ boundary (​The nearest-neighbour is the one incident that is closest to the subject incident than any other incidents; The two events that are the furthest apart while there is no other event that can have a shorter distance with the subject incident​)